#Importação das bibliotecas

In [ ]:
# importo a biblioteca pandas e a twitter
import pandas as pd
import numpy as np
import tweepy
import nltk
import time
import re
import string
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime
import json

from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from dateutil.parser import parse
from dateutil import parser

Instalação da biblioteca do twitter do python

In [ ]:
!pip install tweepy

#Configuração de acesso ao twitter

In [ ]:
env = pd.read_json("environment.json")

consumer_key = env.environment[2]
consumer_secret = env.environment[3]
access_token = env.environment[0]
access_token_secret = env.environment[1]

#Crio a conexão com o twitter
auth = tweepy.OAuthHandler(twitter_consumer_key, twitter_consumer_secret)
auth.set_access_token(twitter_access_token, twitter_access_secret)

api = tweepy.API(auth)

#Funções Auxiliares

In [ ]:
def idade_da_conta(criado_em):   
    ddd = parse(criado_em)
    s = dt.datetime.strftime(ddd,"%Y-%m-%d %H:%M:%S")
    now = dt.datetime.now()
    t = now.strftime("%Y-%m-%d %H:%M:%S")
    date1 = dt.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    date2 = dt.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
    difdate = date2 - date1
    return difdate.days

In [ ]:
def media_de_tweets(quantidade_tweets,criado_em):
    ''' criacao = criado_em
    s = criacao.strftime("%Y-%m-%d %H:%M:%S")
    now = datetime.now()
    t = now.strftime("%Y-%m-%d %H:%M:%S")
    date1 = dt.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    date2 = dt.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
    difdate = date2 - date1
    
    days = difdate.days
    '''
    
    media = quantidade_tweets/criado_em
    return media

In [ ]:
def atributos_gerais(user_id):
    textos = ''
    intervalo_tweets = 0
    dp_intervalo = 0
    dp_dias = 0
    t_segundas = 0
    t_tercas = 0
    t_quartas = 0
    t_quintas = 0
    t_sextas = 0
    t_sabados = 0
    t_domingos = 0
    replies = 0
    mentions = 0
    retweets = 0
    times_retweeted = 0
    tweets = 0
    df_datas = pd.DataFrame()
    datas = []
    favorite_count = 0
    num_hashtags = 0
    num_urls = 0
    i = 0
    df_usuario = df_users_tweets[df_users_tweets['id']  == user_id]
    if len(df_usuario) > 0:
        for index, row in df_usuario.iterrows():
            #if row.in_reply_to_status_id != None: # se o id de resposta for diferente de none
            #    replies = replies + 1
            mentions = mentions + row.num_mentions        
            count = 0
            text = ''
            rt = str(row.text).find("RT", 0, 3)
            if rt != -1:
                retweets = retweets + 1
            else:
                times_retweeted = times_retweeted + row.retweet_count # se alguem retuitou este tweet
            textos += str(row.text)
            textos += ' | '
            tweets = tweets + 1
            datas.append(row.created_at)
            favorite_count = (favorite_count + row.favorite_count if row.favorite_count != None else favorite_count + 0)
            num_hashtags = (num_hashtags + row.num_hashtags if row.num_hashtags != None else num_hashtags + 0)
            num_urls = (num_urls + row.num_urls if row.num_urls != None else num_urls + 0)

            d = row.created_at
            if d.weekday() == 0:
                t_segundas = t_segundas + 1
            elif d.weekday() == 1:
                t_tercas = t_tercas + 1
            elif d.weekday() == 2:
                t_quartas = t_quartas + 1
            elif d.weekday() == 3:
                t_quintas = t_quintas + 1
            elif d.weekday() == 4:
                t_sextas = t_sextas + 1
            elif d.weekday() == 5:
                t_sabados = t_sabados + 1
            elif d.weekday() == 6:
                t_domingos = t_domingos + 1
        dp = [t_segundas,t_tercas,t_quartas,t_quintas,t_sextas,t_sabados,t_domingos]     
        dp_dias = np.std(dp)
        datas.sort()
        df_datas['datas'] = datas
        dif = df_datas["datas"].diff()
        df_datas['diferenca'] = (df_datas['datas']-df_datas['datas'].shift()).fillna(pd.Timedelta(seconds=0))
        intervalo_tweets = round(df_datas['diferenca'].mean().total_seconds())
        dp_intervalo = round(df_datas['diferenca'].std().total_seconds())
        retorno = {"intervalo_tweets": intervalo_tweets,
                   "dp_intervalo": dp_intervalo,
                   "dp_dias": dp_dias,
                   "t_segundas":t_segundas,
                   "t_tercas": t_tercas,
                   "t_quartas": t_quartas,
                   "t_quintas": t_quintas,
                   "t_sextas": t_sextas,
                   "t_sabados": t_sabados,
                   "t_domingos": t_domingos,
                   "posts": textos,
                   "replies": replies,
                   "mentions": mentions,
                   "retweets": retweets,
                   "times_retweeted": times_retweeted,
                   "tweets": tweets,
                   "favorite_count": favorite_count,
                   "num_hashtags": num_hashtags,
                   "num_urls": num_urls,
                  } 
        return retorno

#Leitura do CSV com ids e rótulos

In [ ]:
df_users = pd.read_csv("cresci-rtbust-2019.tsv", sep="\t")
df_users.columns = ['user_id', 'tipo_usuario']
df_users.head()

Verifico a quantidade de amostras

In [ ]:
df_users.shape

Criação da listas que irão receber os resultados das consultas

In [ ]:
ids = []
tipo_usuario = []
name = []
screen_name = []
statuses_count = []
followers_count = []
friends_count = []
favourites_count = []
listed_count = []
url = []
lang = []
time_zone = []
location = []
default_profile = []
default_profile_image = []
geo_enabled = []
profile_image_url = []
#profile_banner_url = []
profile_use_background_image = []
profile_background_image_url_https = []
profile_text_color = []
profile_image_url_https = []
profile_sidebar_border_color = []
profile_background_tile = []
profile_sidebar_fill_color = []
profile_background_image_url = []
profile_background_color = []
profile_link_color = []
utc_offset = []
is_translator = []
follow_request_sent = []
protected = []
verified = []
notifications = []
description = []
contributors_enabled = []
following = []
created_at = []
#timestamp = []
#crawled_at = []
#updated = []
#test_set_2 = []

#Coleta de dados de usuário

In [ ]:
df_dados_usuarios = pd.DataFrame() #defino um dataFrame
count = 1
time.sleep(900)
for index,user in df_users.iterrows():
    try:
        print(user.user_id)
        dados = api.get_user(user_id=user.user_id)
        tipo_usuario.append(user.tipo_usuario)
        ids.append(user.user_id)
        name.append(dados._json["name"])
        screen_name.append(dados._json['screen_name'])
        statuses_count.append(dados._json['statuses_count'])
        followers_count.append(dados._json['followers_count'])
        friends_count.append(dados._json['friends_count'])
        favourites_count.append(dados._json['favourites_count'])
        listed_count.append(dados._json['listed_count'])
        url.append(dados._json['url'])
        lang.append(dados._json['lang'])
        time_zone.append(dados._json['time_zone'])
        location.append(dados._json['location'])
        default_profile.append(dados._json['default_profile'])
        default_profile_image.append(dados._json['default_profile_image'])
        geo_enabled.append(dados._json['geo_enabled'])
        profile_image_url.append(dados._json['profile_image_url'])
        #profile_banner_url.append(dados._json['profile_banner_url'])
        profile_use_background_image.append(dados._json['profile_use_background_image'])
        profile_background_image_url_https.append(dados._json['profile_background_image_url_https'])
        profile_text_color.append(dados._json['profile_text_color'])
        profile_image_url_https.append(dados._json['profile_image_url_https'])
        profile_sidebar_border_color.append(dados._json['profile_sidebar_border_color'])
        profile_background_tile.append(dados._json['profile_background_tile'])
        profile_sidebar_fill_color.append(dados._json['profile_sidebar_fill_color'])
        profile_background_image_url.append(dados._json['profile_background_image_url'])
        profile_background_color.append(dados._json['profile_background_color'])
        profile_link_color.append(dados._json['profile_link_color'])
        utc_offset.append(dados._json['utc_offset'])
        is_translator.append(dados._json['is_translator'])
        follow_request_sent.append(dados._json['follow_request_sent']),
        protected.append(dados._json['protected']),
        verified.append(dados._json['verified']),
        notifications.append(dados._json['notifications']),
        description.append(dados._json['description']),
        contributors_enabled.append(dados._json['contributors_enabled'])
        following.append(dados._json['following'])
        created_at.append(dados._json['created_at'])
        count = count + 1
    except tweepy.TweepError as e:
        print(str(e))
        continue
        if count == 900:
            count = 1
            time.sleep(60 * 15)

Atribuição dos dados coletados ao dataframe

In [ ]:
df_dados_usuarios['id'] = ids # a primeira coluna recebe os usuários
df_dados_usuarios['tipo_usuario'] = tipo_usuario # a primeira coluna recebe os usuários
df_dados_usuarios['name'] = name
df_dados_usuarios['screen_name'] = screen_name
df_dados_usuarios['statuses_count'] = statuses_count
df_dados_usuarios['followers_count'] = followers_count
df_dados_usuarios['friends_count'] = friends_count
df_dados_usuarios['favourites_count'] = favourites_count
df_dados_usuarios['listed_count'] = listed_count
df_dados_usuarios['url'] = url
df_dados_usuarios['lang'] = lang
df_dados_usuarios['time_zone'] = time_zone
df_dados_usuarios['location'] = location
df_dados_usuarios['default_profile'] = default_profile
df_dados_usuarios['default_profile_image'] = default_profile_image
df_dados_usuarios['geo_enabled'] = geo_enabled
df_dados_usuarios['profile_image_url'] = profile_image_url
#df_dados_usuarios['profile_banner_url'] = profile_banner_url
df_dados_usuarios['profile_use_background_image'] = profile_use_background_image
df_dados_usuarios['profile_background_image_url_https'] = profile_background_image_url_https
df_dados_usuarios['profile_text_color'] = profile_text_color
df_dados_usuarios['profile_image_url_https'] = profile_image_url_https
df_dados_usuarios['profile_sidebar_border_color'] = profile_sidebar_border_color
df_dados_usuarios['profile_background_tile'] = profile_background_tile
df_dados_usuarios['profile_sidebar_fill_color'] = profile_sidebar_fill_color
df_dados_usuarios['profile_background_image_url'] = profile_background_image_url
df_dados_usuarios['profile_background_color'] = profile_background_color
df_dados_usuarios['profile_link_color'] = profile_link_color
df_dados_usuarios['utc_offset'] = utc_offset
df_dados_usuarios['is_translator'] = is_translator
df_dados_usuarios['follow_request_sent'] = follow_request_sent
df_dados_usuarios['protected'] = protected
df_dados_usuarios['verified'] = verified
df_dados_usuarios['notifications'] = notifications
df_dados_usuarios['description'] = description
df_dados_usuarios['contributors_enabled'] = contributors_enabled
df_dados_usuarios['following'] = following
df_dados_usuarios['created_at'] = created_at

Verificação de como estão os dados

In [ ]:
df_dados_usuarios.head()

Verificação da quantidade de amostras no dataframe

In [ ]:
df_dados_usuarios.shape

Exportação do conjunto de dados apenas com dados de perfil de usuário

In [ ]:
df_dados_usuarios.to_csv('users_dados_sem_tweets.csv',sep = ';', line_terminator = '', 
                     encoding = 'utf-8', index=False)

#Leitura da base de dados com informações do perfil de usuário

In [ ]:
df_dados_usuarios = pd.read_csv('users_dados_sem_tweets.csv',sep=';')

Verificação da quantidade de amostras

In [ ]:
df_dados_usuarios.shape

Verificação da estrutura do dataframe

In [ ]:
df_dados_usuarios.head()

Verificação das colunas do dataframe

#Declaração de atributos e dataframe que irão receber dados dos tweets

In [ ]:
df_final = pd.DataFrame() #defino um dataFrame

ids = []
tipo_usuario = []
name = []
screen_name = []
statuses_count = []
followers_count = []
friends_count = []
favourites_count = []
listed_count = []
url = []
lang = []
time_zone = []
location = []
default_profile = []
default_profile_image = []
geo_enabled = []
profile_image_url = []
#profile_banner_url = []
profile_use_background_image = []
profile_background_image_url_https = []
profile_text_color = []
profile_image_url_https = []
profile_sidebar_border_color = []
profile_background_tile = []
profile_sidebar_fill_color = []
profile_background_image_url = []
profile_background_color = []
profile_link_color = []
utc_offset = []
is_translator = []
follow_request_sent = []
protected = []
verified = []
notifications = []
description = []
contributors_enabled = []
following = []
created_at = []
num_hashtags = []
num_urls = []
n_mencoes = []
n_vezes_retuitado = []
contagem_favoritos = []
n_respostas = []
n_retweets = []
t_segundas = []
t_tercas = []
t_quartas = []
t_quintas = []
t_sextas = []
t_sabados = []
t_domingos = []
dp_dias = []

Coleta de dados relacionadas aos ultimos 200 tweets de cada usuário

In [ ]:
for index,user in df_dados_usuarios.iterrows(): # pra cada pego os seus ultimos 200 posts
    try:
        numhash = 0
        nummentions = 0
        numurls = 0
        numretuitado = 0
        numfavoritado = 0
        numrespostas = 0
        numretuites = 0

        segundas = 0
        tercas = 0
        quartas = 0
        quintas = 0
        sextas = 0
        sabados = 0
        domingos = 0

        intervalo_tweets = 0
        dp_intervalo = 0
        desviop_dias = 0
        df_datas = pd.DataFrame()
        datas = []

        for status in tweepy.Cursor(api.user_timeline, id=user.id).items(200):
            numhash = numhash + len(status._json['entities']['hashtags'])
            nummentions = nummentions + len(status._json['entities']['user_mentions'])
            numurls = numurls + len(status._json['entities']['urls'])
            numretuitado = numretuitado + status._json['retweet_count']
            numfavoritado = numfavoritado + status._json['favorite_count']    
            if status._json['in_reply_to_status_id'] != None: # se o id de resposta for diferente de none
                numrespostas = numrespostas + 1
            if math.isnan(pd.isna(status.text)) == True:
                rt = status._json['text'].find("RT", 0, 3)
                if rt != -1:
                    numretuites = numretuites + 1
            d = parser.parse(status._json['created_at'])
            datas.append(d)
            #print(d)
            if d.weekday() == 0:
                segundas = segundas + 1
            elif d.weekday() == 1:
                tercas = tercas + 1
            elif d.weekday() == 2:
                quartas = quartas + 1
            elif d.weekday() == 3:
                quintas = quintas + 1
            elif d.weekday() == 4:
                sextas = sextas + 1
            elif d.weekday() == 5:
                sabados = sabados + 1
            elif d.weekday() == 6:
                domingos = domingos + 1

        dp = [segundas,tercas,quartas,quintas,sextas,sabados,domingos]     
        desviop_dias = np.std(dp)
        datas.sort()
        
        tipo_usuario.append(user.tipo_usuario)
        ids.append(user.id)
        name.append(user.name)
        screen_name.append(user.screen_name)
        statuses_count.append(user.statuses_count)
        followers_count.append(user.followers_count)
        friends_count.append(user.friends_count)
        favourites_count.append(user.favourites_count)
        listed_count.append(user.listed_count)
        url.append(user.url)
        lang.append(user.lang)
        time_zone.append(user.time_zone)
        location.append(user.location)
        default_profile.append(user.default_profile)
        default_profile_image.append(user.default_profile_image)
        geo_enabled.append(user.geo_enabled)
        profile_image_url.append(user.profile_image_url)
        #profile_banner_url.append(dados._json['profile_banner_url'])
        profile_use_background_image.append(user.profile_use_background_image)
        profile_background_image_url_https.append(user.profile_background_image_url_https)
        profile_text_color.append(user.profile_text_color)
        profile_image_url_https.append(user.profile_image_url_https)
        profile_sidebar_border_color.append(user.profile_sidebar_border_color)
        profile_background_tile.append(user.profile_background_tile)
        profile_sidebar_fill_color.append(user.profile_sidebar_fill_color)
        profile_background_image_url.append(user.profile_background_image_url)
        profile_background_color.append(user.profile_background_color)
        profile_link_color.append(user.profile_link_color)
        utc_offset.append(user.utc_offset)
        is_translator.append(user.is_translator)
        follow_request_sent.append(user.follow_request_sent)
        protected.append(user.protected)
        verified.append(user.verified)
        notifications.append(user.notifications)
        description.append(user.description)
        contributors_enabled.append(user.contributors_enabled)
        following.append(user.following)
        created_at.append(user.created_at)

        num_hashtags.append(numhash)
        num_urls.append(numurls)
        n_mencoes.append(nummentions)
        n_vezes_retuitado.append(numretuitado)
        contagem_favoritos.append(numfavoritado)
        n_respostas.append(numrespostas)
        n_retweets.append(numretuites)
        t_segundas.append(segundas)
        t_tercas.append(tercas)
        t_quartas.append(quartas)
        t_quintas.append(quintas)
        t_sextas.append(sextas)
        t_sabados.append(sabados)
        t_domingos.append(domingos)
        dp_dias.append(dp_dias)
    except tweepy.TweepError as e:
        print(str(e))
        time.sleep(60 * 3)
        continue

Associação dos dados as variaveis

In [ ]:
df_final['id'] = ids # a primeira coluna recebe os usuários
df_final['tipo_usuario'] = tipo_usuario # a primeira coluna recebe os usuários
df_final['name'] = name
df_final['screen_name'] = screen_name
df_final['statuses_count'] = statuses_count
df_final['followers_count'] = followers_count
df_final['friends_count'] = friends_count
df_final['favourites_count'] = favourites_count
df_final['listed_count'] = listed_count
df_final['url'] = url
df_final['lang'] = lang
df_final['time_zone'] = time_zone
df_final['location'] = location
df_final['default_profile'] = default_profile
df_final['default_profile_image'] = default_profile_image
df_final['geo_enabled'] = geo_enabled
df_final['profile_image_url'] = profile_image_url
#df_final['profile_banner_url'] = profile_banner_url
df_final['profile_use_background_image'] = profile_use_background_image
df_final['profile_background_image_url_https'] = profile_background_image_url_https
df_final['profile_text_color'] = profile_text_color
df_final['profile_image_url_https'] = profile_image_url_https
df_final['profile_sidebar_border_color'] = profile_sidebar_border_color
df_final['profile_background_tile'] = profile_background_tile
df_final['profile_sidebar_fill_color'] = profile_sidebar_fill_color
df_final['profile_background_image_url'] = profile_background_image_url
df_final['profile_background_color'] = profile_background_color
df_final['profile_link_color'] = profile_link_color
df_final['utc_offset'] = utc_offset
df_final['is_translator'] = is_translator
df_final['follow_request_sent'] = follow_request_sent
df_final['protected'] = protected
df_final['verified'] = verified
df_final['notifications'] = notifications
df_final['description'] = description
df_final['contributors_enabled'] = contributors_enabled
df_final['following'] = following
df_final['created_at'] = created_at


df_final['id'] = ids # a primeira coluna recebe os usuários
df_final['tipo_usuario'] = tipo_usuario # a primeira coluna recebe os usuários
df_final["num_hashtags"] = num_hashtags
df_final["num_urls"] = num_urls
df_final['n_mencoes'] = n_mencoes
df_final['n_vezes_retuitado'] = n_vezes_retuitado 
df_final['contagem_favoritos'] = contagem_favoritos # a segunda coluna os seus tweets
df_final['n_respostas'] = n_respostas
df_final['n_retweets'] = n_retweets
df_final['t_segundas'] = t_segundas
df_final['t_tercas'] = t_tercas
df_final['t_quartas'] = t_quartas
df_final['t_quintas'] = t_quintas
df_final['t_sextas'] = t_sextas
df_final['t_sabados'] = t_sabados
df_final['t_domingos'] = t_domingos
df_final['dp_dias'] = dp_dias

Verificação da estrutura do dataframe final

In [ ]:
df_final.head()

Exportação do conjunto de contexto aberto

In [ ]:
df_final.to_csv('base_validacao_final.csv',sep = ';', line_terminator = '', 
                     encoding = 'utf-8', index=False)